In [1]:
!nvidia-smi

Sat Jun 11 04:47:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q transformers
!pip install -q datasets
!pip install -q seqeval
# !pip install -q gradio

     |████████████████████████████████| 4.2 MB 16.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 346 kB 17.2 MB/s 
     |████████████████████████████████| 212 kB 69.5 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 66.5 MB/s 
     |████████████████████████████████| 140 kB 72.7 MB/s 
     |████████████████████████████████| 127 kB 64.5 MB/s 
     |████████████████████████████████| 271 kB 75.7 MB/s 
     |████████████████████████████████| 144 kB 64.3 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 112 kB 75.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 

In [3]:
# # Install the vncorenlp python wrapper
# !pip3 install vncorenlp

# # Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 15.6 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=e1710cd2cc2ddaac8f59e2ba775d47f08db5737d696fc27914e0fdf0cccdb078
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
--2022-06-11 04:47:42--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2022-0

In [4]:
import os
import sys

import pandas as pd
import numpy as np
import datasets
from vncorenlp import VnCoreNLP

from datasets import Dataset
from tqdm.notebook import tqdm
from nltk.corpus.reader import ConllCorpusReader

import transformers
from transformers import AutoModelForTokenClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification

from datasets import Features, Sequence, ClassLabel, Value, DatasetDict
from datasets import load_metric

In [5]:
labels = ['B-PATIENT_ID', 'I-PATIENT_ID',
          'B-NAME', 'I-NAME',
          'B-AGE', 'I-AGE',
          'B-GENDER',
          'B-JOB', 'I-JOB',
          'B-LOCATION', 'I-LOCATION',
          'B-ORGANIZATION', 'I-ORGANIZATION',
          'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE',
          'B-TRANSPORTATION', 'I-TRANSPORTATION',
          'B-DATE', 'I-DATE',
          'O']

id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
id2label

{0: 'B-PATIENT_ID',
 1: 'I-PATIENT_ID',
 2: 'B-NAME',
 3: 'I-NAME',
 4: 'B-AGE',
 5: 'I-AGE',
 6: 'B-GENDER',
 7: 'B-JOB',
 8: 'I-JOB',
 9: 'B-LOCATION',
 10: 'I-LOCATION',
 11: 'B-ORGANIZATION',
 12: 'I-ORGANIZATION',
 13: 'B-SYMPTOM_AND_DISEASE',
 14: 'I-SYMPTOM_AND_DISEASE',
 15: 'B-TRANSPORTATION',
 16: 'I-TRANSPORTATION',
 17: 'B-DATE',
 18: 'I-DATE',
 19: 'O'}

In [6]:
bert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def process_data(text):
    sentences = rdrsegmenter.tokenize(text)
    segmented_text = ""
    firstSent = True
    for sentence in sentences:
        if firstSent:
            segmented_text += " ".join(sentence)
            firstSent = False
        else:
            segmented_text += " "
            segmented_text += " ".join(sentence)
    
    tokenized_input = bert_tokenizer.tokenize(segmented_text)

    input = bert_tokenizer(segmented_text,
                           add_special_tokens=True,
                           return_attention_mask=True,
                           return_token_type_ids=False,
                           return_tensors='pt')
    
    return tokenized_input, input

In [8]:
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP/learning/NLP-K31/pretrained/phoBert_large_mlm_task", num_labels=len(labels))
model.cuda().eval()

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 1024, padding_idx=1)
      (position_embeddings): Embedding(258, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
            

In [9]:
def inference(text):

    tokenized_input, input = process_data(text)
    input_ids = input['input_ids'].cuda()
    attention_mask = input['attention_mask'].cuda()
    logits = model(input_ids).logits.cpu().detach().numpy()
    predictions = np.argmax(logits, axis=2)[0][1:-1]

    output = ""
    i = 0
    while (i < len(predictions)):
        if tokenized_input[i].endswith('@@'):
            tmp_inp = ""
            label = id2label[predictions[i]]
            while tokenized_input[i].endswith('@@'):
                tmp_inp += tokenized_input[i].strip('@@')
                i += 1
            tmp_inp += tokenized_input[i]
            output += tmp_inp + '\t\t' + label + '\n'
            i += 1
        
        else:
            output += tokenized_input[i] + '\t\t' + id2label[predictions[i]] + '\n'
            i += 1
    
    return output

In [10]:
text = "Từ 24 - 7 đến 31 - 7, bệnh nhân được mẹ là bà H.T.P (47 tuổi) đón về nhà ở phường Phước Hòa (bằng xe máy), không đi đâu chỉ ra tạp hóa Phượng, chợ Vườn Lài, phường An sơn cùng mẹ bán tạp hóa ở đây."

In [12]:
print(inference(text))

Từ		O
24		B-DATE
-		I-DATE
7		I-DATE
đến		O
31		B-DATE
-		I-DATE
7		I-DATE
,		O
bệnh_nhân		O
được		O
mẹ		O
là		O
bà		O
H.T.P		B-NAME
(		O
47		B-AGE
tuổi		O
)		O
đón		O
về		O
nhà		O
ở		O
phường		B-LOCATION
Phước_Hoà		I-LOCATION
(		O
bằng		O
xe_máy		O
)		O
,		O
không		O
đi		O
đâu		O
chỉ		O
ra		O
tạp_hoá		B-LOCATION
Phượng		I-LOCATION
,		O
chợ		B-LOCATION
Vườn_Lài		I-LOCATION
,		O
phường		B-LOCATION
An		I-LOCATION
sơn		I-LOCATION
cùng		O
mẹ		O
bán		O
tạp_hoá		O
ở		O
đây		O
.		O

